### Es 1
Hai a disposizione un file `data.csv` contenente dati mensili di passeggeri con due colonne:

- `date`: data in formato `YYYY-MM` (mese/anno)
- `passengers`: numero di passeggeri per quel mese


Costruisci un modello di **regressione polinomiale** che approssima l’andamento del numero di passeggeri nel tempo.

1. Carica il dataset.
2. Convertilo in un formato numerico utilizzando una colonna `mese_numerico` che conti i mesi a partire da gennaio 1949.
3. Applica una regressione polinomiale (grado a tua scelta).
4. Calcola l’RMSE tra i valori reali e quelli predetti.
5. Visualizza i dati reali e la curva stimata con Plotly.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

#1 carica il dataset
df = pd.read_csv('data.csv')

#2 - Aggiungo colonna mese_numerico
df['date'] = pd.to_datetime(df['date'])
df['mese_numerico']= (df['date'].dt.year - 1949) * 12 + df['date'].dt.month

#3 - Regressione polinomiale di grado 4
grado = 4
x = df[['mese_numerico']]

# Sostituisco i valori NaN con la media della colonna 'passengers'
y = df['passengers'].fillna(df['passengers'].mean())



poly = PolynomialFeatures(degree=grado)
x_poly = poly.fit_transform(x)

model = LinearRegression()
model.fit(x_poly, y)
y_pred = model.predict(x_poly)

#4 - Calcolo RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f"RMSE: {rmse:.2f}")

#5 - Grafico con utilizzo di plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['date'], y=y, mode='lines+markers', name='Dati Reali'))
fig.add_trace(go.Scatter(x=df['date'], y=y_pred, mode='lines', name='Previsione Polinomiale'))
fig.update_layout(title='Previsione del numero di passeggeri',
                  xaxis_title='Data',
                  yaxis_title='Numero di passeggeri',
                  legend_title='Legenda')
fig.show()


RMSE: 47.64


### Es2. 
Costruisci una web app con Dash che permette all’utente di scegliere il grado del polinomio per adattare un modello di regressione ai dati non lineari e vedere il risultato aggiornarsi dinamicamente.


1. Genera 100 punti x tra -3 e 3.

2. Calcola ad esempio y = x³ - x + rumore.

3. Costruisci un'interfaccia Dash con:
    - uno slider per scegliere il grado del polinomio (1–10),
    - un grafico Plotly che mostra i dati e la curva stimata.

4. Usa PolynomialFeatures + LinearRegression da scikit-learn per stimare la curva

In [5]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

step = 100

x = np.linspace(-3, 3, step) #Creo un array con valori da -3 a 3 step 100
rumore = np.random.normal(loc=0, scale=0.1, size=len(x)) #Creo un array di rumore gaussiano
y = x**3 - x + rumore



#Creo un dataframe per visualizzare i dati e poterli plottare
df = pd.DataFrame({'x': x, 'y': y})
#print(df)

#Costruzione dell'interfaccia dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2('Regressione Polinomiale con Dash'),

    dcc.Slider(
        id='degree-slider',
        min=1,
        max=10,
        step=1,
        value=4,
        marks={i: str(i) for i in range(1, 11)},
    ),

    html.Div(id='Polynomial-degree-output'),
    dcc.Graph(id='Polynomial-degree-graph')
])

@app.callback(
    Output('Polynomial-degree-graph', 'figure'),
    Output('Polynomial-degree-output', 'children'),
    Input('degree-slider', 'value')
)
def update_output(degree):
    x = df['x'].values.reshape(-1, 1)
    y = df['y'].values

    # Regressione polinomiale
    poly = PolynomialFeatures(degree=degree)
    x_poly = poly.fit_transform(x)

    model = LinearRegression()
    model.fit(x_poly, y)

    y_pred = model.predict(x_poly)

    # Grafico
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['x'], y=df['y'], mode='markers', name='Dati Reali'))
    fig.add_trace(go.Scatter(x=df['x'], y=y_pred, mode='lines', name='Previsione Polinomiale'))
    fig.update_layout(title=f'Regressione Polinomiale di grado {degree}',
                      xaxis_title='x',
                      yaxis_title='y',
                      legend_title='Legenda')
    
    return fig, f'Degree: {degree}'

if __name__ == '__main__':
    app.run_server(debug=True) #http://127.0.0.1:8050